In [14]:
using GLMakie
using LaTeXStrings
using Logging
using NBInclude


# Apagamos los warnings con Julia
Logging.disable_logging(Logging.Info)

LogLevel(1)

# Modelo de una compuerta

## Cambio de estado

La probabilidad de que una compuerta esté abierta, como función del tiempo, es una combinación lineal de las probabilidades de que la compuerta se halla cerrado o abierto y las reglas observadas de su comportamiento se describen con una ecuación diferencial.

Este comportamiento se ve afectado por el voltaje de polarización de la membrana, pero para esta sección consideraremos al voltaje como un parámetro fijo, de modo que nos podamos enfocar en la ecuación como función del tiempo.  Tomemos como ejemplo la ecuación para la compuerta de potasio (K).

El cambio en la probabilidad de que una compuerta esté abierta se puede decir que es proporcional a:

\begin{align*}
 \Delta \text{Compuertas abiertas} = \text{Puertas que se abren} - \text{Puertas que se cierran}
\end{align*}

Formalizando, es la suma de la probabilidad de que la compuerta esté cerrada por la probabilidad de que se abra, menos la probabilidad de que haya estado abierta y se cierre. Esto se expresa con una ecuación diferencial:

\begin{equation}
 \frac{1}{\gamma(T)} \frac{dn}{dt} = \alpha_n(V) (1 - n) - \beta_n(V)n
\end{equation}

donde $\gamma(T)$ es una constante de proporcionalidad temporal que depende de la temperatura $T$, **a $6.3°$C vale $1$** [Cessac2009].

### Estado de equilibrio

La ecuación anterior es una ecuación diferencial de primer grado, si bien la ecuación describe la variación de $n$ en el tiempo, existe un punto especial en el cual $n$ ya no cambia.  A éste se le llama un **estado de equilibrio**, matemáticamente corresponde a que su derivada sea cero.

\begin{align}
  \frac{dn}{dt} &= 0
\end{align}

Si sustituimos esta condición en la ecuación anterior obtenemos el valor que toma $n$ al alcanzar el equilibrio, al cual llamaremos $n_{\infty}$; esto como función del voltaje de polarización actual $V$:
\begin{align}
 \alpha_n(V) (1 - n_{\infty}) - \beta_n(V)n_{\infty} &= 0 \\
 n_{\infty}(V) &= \frac{\alpha_n(V)}{\alpha_n(V) + \beta_n(V)}
\end{align}


### Constante temporal

Se define ahora a la constante temporal:
\begin{align*}
 \tau_n(V) &= \frac{1}{\alpha_n(V) + \beta_n(V)}
\end{align*}

In [23]:
@nbinclude("funciones_de_transicion.ipynb")

In [24]:
"""
Grafica alfa y beta como función del voltaje.
"""
function plot_parameters()
    fig = Figure()
    V = -100:0.1:100

    alpha = alpha_n.(V)
    beta = beta_n.(V)

end

plot_parameters

### Ecuación de transición

Es costumbre reescribir la ecuación diferencial original en términos de $n_{\infty}$ y $\tau_n$:

\begin{align}
 \frac{1}{\gamma(T)} \frac{dn}{dt} &= \frac{n^{\infty}(V) - n}{\tau_n(V)}
\end{align}

## Solución numérica

Para obtener el valor de $n$ al tiempo $t$ dado un voltaje fijo $V$ utlizaremos el método de **Euler** para resolver numéricamente ecuaciones diferenciales de primer orden.  Si bien existe una solución simbólica para este problema, no nos servirá cuando queramos recuperar el hecho de que el voltaje no permanece constante.

### Método de Euler para la ecuación de la compuerta

Aplicando esta fórmula a la ecuación para el canal de potasio:

\begin{align}
 \frac{dn}{dt} &= \frac{n^{\infty}(V) - n(t)}{\tau_n(V)}
\end{align}

Obtenemos:

\begin{align}
 n(t + \Delta t) =& n(t) + \Delta t \left[\frac{n^{\infty}(V) - n(t)}{\tau_n(V)} \right] \\
                 =& \left[ 1 - \frac{\Delta t}{\tau_n(V)} \right] n(t) + \frac{\Delta t}{\tau_n(V)}n^{\infty}(V)
\end{align}

# Implementación

In [25]:
"""
Aplicación del método de Euler para aproximar n(t) cuando
el voltaje es fijo.
"""
function compuerta_de_potasio(n_0, voltaje)
    delta = 0.01
    max_t = 50.0

    t = 0:delta:max_t
    n = zeros(length(t))

    # Los siguientes valores son constantes sólo si el voltaje
    # es constante. Si no lo es, deben ser calculados para cada t
    # como función de V(t).

    α = alpha_n.(voltaje)
    β = beta_n.(voltaje)
    t = tau_inf.(α, β)
    n_i = n_inf.(α, β)

    n[1] = n_0 # Valor inicial propuesto
    for i in 1:length(t)-1
        n[i+1] = (1 - d / t) * n[i] + d * n_i / t
    end

    return Dict("t" => t, "n" => n)
end

compuerta_de_potasio

In [33]:
"""
Grafica n(t) vs t según fue aproximada por Euler.
"""
function plot_potasio(resultado::Dict)
    tiempo = resultado["t"]

    fig = Figure()
    ax = Axis(fig[1, 1])

    lines!(ax, tiempo, resultado["n"], color = :blue)
    xlabel!("Tiempo (ms)")
    ylabel!("n(t) (asimensional))")
    title!(L"\textit{Tasa de transición } n(t)")

    fig
end

plot_potasio

In [34]:
# Resultado para el punto inicial (n = 0, V = 20 mV)
plot_potasio(compuerta_de_potasio(0, 20))

LoadError: `Makie.convert_arguments` for the plot type Lines and its conversion trait PointBased() was unsuccessful.

The signature that could not be converted was:
::Float64, ::Vector{Float32}

Makie needs to convert all plot input arguments to types that can be consumed by the backends (typically Arrays with Float32 elements).
You can define a method for `Makie.convert_arguments` (a type recipe) for these types or their supertypes to make this set of arguments convertible (See http://docs.makie.org/stable/documentation/recipes/index.html).

Alternatively, you can define `Makie.convert_single_argument` for single arguments which have types that are unknown to Makie but which can be converted to known types and fed back to the conversion pipeline.
